In [22]:
import pandas as pd
import numpy as np

In [50]:
df_train = pd.read_csv('../data/train.csv')
#df_test = pd.read_csv('../data/test.csv')

In [3]:
df_train.columns.values

array(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion',
       'ciudad', 'provincia', 'antiguedad', 'habitaciones', 'garages',
       'banos', 'metroscubiertos', 'metrostotales', 'idzona', 'lat',
       'lng', 'fecha', 'gimnasio', 'usosmultiples', 'piscina',
       'escuelascercanas', 'centroscomercialescercanos', 'precio'],
      dtype=object)

In [4]:
df_train.ciudad.value_counts()

Querétaro          12646
Benito Juárez      11014
Zapopan            10360
San Luis Potosí     7925
Mérida              7162
                   ...  
Chilchotla             1
Abalá                  1
Zimapán                1
Rayones                1
Ocampo                 1
Name: ciudad, Length: 875, dtype: int64

In [5]:
df_train.provincia.value_counts()

Distrito Federal         58790
Edo. de México           41607
Jalisco                  21238
Querétaro                16988
Nuevo León               15324
Puebla                   10421
San luis Potosí           8447
Yucatán                   7928
Morelos                   7337
Veracruz                  5762
Quintana Roo              4756
Chihuahua                 4590
Coahuila                  3695
Baja California Norte     3220
Sonora                    2988
Guanajuato                2860
Guerrero                  2678
Hidalgo                   2521
Michoacán                 2471
Tamaulipas                2303
Durango                   2275
Sinaloa                   1806
Aguascalientes            1753
Baja California Sur       1700
Nayarit                   1352
Chiapas                   1126
Colima                    1008
Tabasco                    994
Tlaxcala                   839
Oaxaca                     711
Campeche                   263
Zacatecas                   94
Name: pr

In [6]:
df_train.tipodepropiedad.value_counts()

Casa                             141717
Apartamento                       57341
Casa en condominio                19297
Terreno                            9945
Local Comercial                    3055
Oficina comercial                  1741
Bodega comercial                   1406
Edificio                           1396
Terreno comercial                  1326
Casa uso de suelo                   708
Quinta Vacacional                   395
Duplex                              343
Villa                               340
Inmuebles productivos urbanos       200
Rancho                              170
Local en centro comercial           165
Departamento Compartido             141
Otros                               134
Nave industrial                      76
Terreno industrial                   31
Huerta                               20
Lote                                  5
Garage                                1
Hospedaje                             1
Name: tipodepropiedad, dtype: int64

In [7]:
df_train.lng.value_counts()

 0.000000      449
-99.225000      12
-99.218984       9
-99.139000       8
-99.213000       8
              ... 
-103.449196      1
-100.443135      1
-106.103384      1
-99.168257       1
-99.131383       1
Name: lng, Length: 112326, dtype: int64

In [8]:
df_train.fecha.value_counts()

2016-12-03 00:00:00    1416
2016-04-08 00:00:00    1337
2016-12-28 00:00:00    1182
2012-08-17 00:00:00    1168
2016-12-27 00:00:00    1167
                       ... 
2012-12-24 00:00:00      11
2013-02-10 00:00:00       9
2012-12-30 00:00:00       5
2014-09-30 18:12:16       2
2014-08-31 18:12:16       1
Name: fecha, Length: 1830, dtype: int64

In [9]:
df_train = pd.read_csv('../data/train.csv')
df_train = df_train[df_train['tipodepropiedad']=='Casa'].dropna()
df_train['patio'] = df_train['metrostotales'] - df_train['metroscubiertos']
df_train = df_train[df_train['patio'] >= 0].dropna()

df_train.patio.value_counts()

0.0      2922
20.0      684
10.0      522
30.0      478
40.0      407
         ... 
311.0       1
284.0       1
278.0       1
193.0       1
327.0       1
Name: patio, Length: 287, dtype: int64

In [94]:
df_train = pd.read_csv('../data/train.csv')

# Feature Engineering (agrego datos de densidad poblacional y criminalidad a las provincias)
provincia_info = {
    'provincia': ["Aguascalientes", "Baja California Norte", "Baja California Sur", "Campeche", "Coahuila", "Colima", "Chiapas", "Chihuahua", "Distrito Federal", "Durango", "Guanajuato", "Guerrero", "Hidalgo", "Jalisco", "Edo. de México", "Michoacán", "Morelos", "Nayarit", "Nuevo León", "Oaxaca", "Puebla", "Querétaro", "Quintana Roo", "San luis Potosí", "Sinaloa", "Sonora", "Tabasco", "Tamaulipas", "Tlaxcala", "Veracruz", "Yucatán", "Zacatecas"],
    'densidadpoblacional': [233.7, 46.4, 9.6, 15.6, 19.5, 126.4, 71.2, 14.4, 5967.3, 14.2, 191.3, 55.6, 137.3, 99.8, 724.2, 78.2, 390.2, 42.4, 79.8, 42.3, 179.8, 174.4, 33.6, 44.5, 51.7, 15.9, 96.9, 42.9, 318.4, 113.0, 53.1, 21.0],
    'criminalidad': [35.457, 32.758, 25.577, 22.114, 24.800, 27.045, 16.687, 31.274, 52.718, 25.640, 33.154, 53.875, 21.159, 49.317, 56.835, 23.876, 43.419, 21.288, 26.221, 24.961, 27.530, 30.991, 35.639, 25.838, 22.750, 40.466, 30.409, 21.363, 30.699, 22.157, 25.862, 21.501]
}

df_provincia_info = pd.DataFrame(provincia_info, columns = ['provincia', 'densidadpoblacional', 'criminalidad'])

df_train = pd.merge(df_train, df_provincia_info, on = "provincia")

# Agrego longitud de la descripcion

def amenitiesCounter(descripcion):
    words = descripcion.lower().split()
    
    amenities = [
        'internet' in words,
        'televisión' in words or 'televisores' in words,
        'estufa' in words or 'estufas' in words,
        'aire' in words and 'acondicionado' in words,
        'balcón' in words or 'balcon' in words,
        'asador' in words,
        'jardin' in words or 'jardín' in words,
        'jacuzzi' in words,
    ]
        
    return len([ x for x in amenities if x])
    
df_train['comodidades'] = df_train['descripcion'].apply(lambda x: amenitiesCounter(str(x)))


print('Total Original: ', df_train.shape[0])

# Filtro por tipo de propiedad
df_train = df_train[df_train['tipodepropiedad'] == 'Casa'].dropna()

print('Solo casas: ', df_train.shape[0])

# Filtro por provincias
provincias = ["Aguascalientes", "Baja California Norte", "Baja California Sur", "Campeche", "Coahuila", "Colima", "Chiapas", "Chihuahua", "Distrito Federal", "Durango", "Guanajuato", "Guerrero", "Hidalgo", "Jalisco", "Edo. de México", "Michoacán", "Morelos", "Nayarit", "Nuevo León", "Oaxaca", "Puebla", "Querétaro", "Quintana Roo", "San luis Potosí", "Sinaloa", "Sonora", "Tabasco", "Tamaulipas", "Tlaxcala", "Veracruz", "Yucatán", "Zacatecas"]
df_train = df_train[df_train['provincia'].isin(provincias)]

print('Algunas provincias: ', df_train.shape[0])

# Filtro por precio
lower = np.percentile(df_train['precio'], 30)
upper = np.percentile(df_train['precio'], 70)

df_train = df_train[df_train['precio'].between(lower, upper)]

print('Precios acotados: ', df_train.shape[0])

# Filtro por metros
lower = np.percentile(df_train['metrostotales'], 30)
upper = np.percentile(df_train['metrostotales'], 70)

df_train = df_train[df_train['metrostotales'].between(lower, upper)]

print('Metros totales acotados: ', df_train.shape[0])

# Filtro por densidad poblacional
lower = np.percentile(df_train['densidadpoblacional'], 30)
upper = np.percentile(df_train['densidadpoblacional'], 70)

df_train = df_train[df_train['densidadpoblacional'].between(lower, upper)]

print('Densidad poblacional acotada: ', df_train.shape[0])

# Filtro por antiguedad
upper = np.percentile(df_train['antiguedad'], 80)

df_train = df_train[df_train['antiguedad'].between(0, upper)]

print('Antiguedad acotada: ', df_train.shape[0])




print("comodidades", df_train[df_train['comodidades'] != 0].dropna().shape[0])

df_train[['id','provincia', 'precio', 'densidadpoblacional', 'criminalidad', 'antiguedad', 'comodidades']]

Total Original:  239845
Solo casas:  32816
Algunas provincias:  32816
Precios acotados:  13240
Metros totales acotados:  6167
Densidad poblacional acotada:  4194
Antiguedad acotada:  3361
comodidades 810


,id,provincia,precio,densidadpoblacional,criminalidad,antiguedad,comodidades
58856,157965,Jalisco,1600000.0,99.8,49.317,5.0,0
58940,152758,Jalisco,1000000.0,99.8,49.317,10.0,0
58962,164710,Jalisco,2350000.0,99.8,49.317,10.0,0
58992,111273,Jalisco,1650000.0,99.8,49.317,5.0,0
58998,150850,Jalisco,1680000.0,99.8,49.317,5.0,1
...,...,...,...,...,...,...,...
239380,297720,Tlaxcala,1650000.0,318.4,30.699,0.0,1
239400,140397,Tlaxcala,1400000.0,318.4,30.699,8.0,0
239466,216779,Tlaxcala,1700000.0,318.4,30.699,1.0,0
239467,181323,Tlaxcala,1150000.0,318.4,30.699,0.0,0


In [90]:
df_test.to_csv('../data/test1.csv', index=False)
df_train.to_csv('../data/train1.csv', index=False)
